In [1]:
### Using Darts package to generate covariates (of course we can do manually)
from darts.utils.timeseries_generation import datetime_attribute_timeseries, holidays_timeseries, linear_timeseries
from darts import TimeSeries, concatenate
from darts.dataprocessing.transformers import Scaler
from darts.utils.missing_values import fill_missing_values
import darts
print('darts version:', darts.__version__)

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 1000)
from datetime import datetime, timedelta
import shutil

import warnings
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

DLNAME='tft_google'
TARGET='rptorder_aov'

## Choosing workspace (which has GPU machine)
DGX=True
Databricks=False
AutoDL=False
Colab=False 
KKlaptop=False
GTX1080=False

## Set working directories 
if GTX1080:
    ### spec: 4x GeForce GTX 1080 Ti
    CURRENT_PATH = '/home/zhaoqi/cbcv'
    READ_DIR = '/home/zhaoqi/cbcv/data/weekly_cohort_sample100_zeromasked'

if DGX:
    ### spec: 8x Tesla V100
    CURRENT_PATH = '/home/jhan405/cbcv/tft_google_ver4/jianlin-tft'
    READ_DIR = f'{CURRENT_PATH}/data/weekly_cohort_data_1000'

if Databricks:
    ### cluster
    pass

if AutoDL:
    CURRENT_PATH = '/root/autodl-tmp/Boston'
    READ_DIR = f'{CURRENT_PATH}/data/weekly_cohort_sample100_zeromasked'


### Set where to save
SAVE_DIR =f'{CURRENT_PATH}/data/preprocessed_data/{DLNAME}/{TARGET}' # where to save prediction results


## Create a new directory if it does not exist
import os
if not os.path.exists(SAVE_DIR):
    os.makedirs(SAVE_DIR)


darts version: 0.20.0


In [2]:
INPUT_CHUNK_LENGTH = 12

## set date range and time unit
TRAIN_START='2017-01-01' # Q1 2017
TEST_START = '2019-04-01' # Q2 2019
TRAIN_START_INPUTCHUNK=(pd.to_datetime(TRAIN_START) - timedelta(days=7*INPUT_CHUNK_LENGTH)).strftime('%Y-%m-%d')
TRAIN_END=(pd.to_datetime(TEST_START) - timedelta(days=1)).strftime('%Y-%m-%d')
TEST_END='2020-02-29' # Before Covid
TEST_END_EXTEND='2020-05-31' # + additional one quarter (for computational issue)

In [3]:
### read filelist
import glob
filelist=glob.glob(f'{READ_DIR}/*') # all paths for data files
filelist.sort()

K_LIST = range(len(filelist))
print(K_LIST)
print(filelist[:5])


range(0, 1075)
['/home/jhan405/cbcv/tft_google_ver4/data/weekly_cohort_data_1000/1000_jam_city_inc_jam_city_inc_cohort.csv', '/home/jhan405/cbcv/tft_google_ver4/data/weekly_cohort_data_1000/1001_heritage_dairy_heritage_dairy_cohort.csv', '/home/jhan405/cbcv/tft_google_ver4/data/weekly_cohort_data_1000/1002_unitedhealth_optumrx_cohort.csv', '/home/jhan405/cbcv/tft_google_ver4/data/weekly_cohort_data_1000/1003_h_r_block_h_r_diy_cohort.csv', '/home/jhan405/cbcv/tft_google_ver4/data/weekly_cohort_data_1000/1004_bmw_payments_bmw_payments_cohort.csv']


In [4]:
def read_file(K):

    ## choose company
    COMPANY_NAME = os.path.splitext(os.path.split(filelist[K])[1])[0][:-len('_cohort')]
    FILEPATH = os.path.splitext(os.path.split(filelist[K])[1])[0] + '.csv'
    print('K =', K, ', COMPANY_NAME:', COMPANY_NAME)

    ## read company
    COL_NAMES_TO_READ = ['acq_week', # group identifier
                    'week', 'tenure', # time identifier
                    'spend', 'orders', # target
                    'N_week_cohort', 'acq_quarter', 'merchant_index', 'merchant'] 

    raw_df = pd.read_csv(f'{READ_DIR}/{FILEPATH}') 
                    # usecols=COL_NAMES_TO_READ)[COL_NAMES_TO_READ]

    # create initial spend
    raw_df['initial_order'] = raw_df['orders'] - raw_df['rpt_orders']
    raw_df['initial_spend'] = raw_df['spend'] - raw_df['rpt_spend']

    # aov
    raw_df['aov'] = raw_df['spend'] / raw_df['orders']
    raw_df['initial_aov'] = raw_df['initial_spend'] / raw_df['initial_order']
    raw_df['rpt_aov'] = raw_df['rpt_spend'] / raw_df['rpt_orders']

    # create per customer
    raw_df['orders_per_cust'] = raw_df['orders'] / raw_df['N_week_cohort']
    raw_df['initial_order_per_cust'] = raw_df['initial_order'] / raw_df['N_week_cohort']
    raw_df['rpt_orders_per_cust'] = raw_df['rpt_orders'] / raw_df['N_week_cohort']
    raw_df['spend_per_cust'] = raw_df['spend'] / raw_df['N_week_cohort']
    raw_df['initial_spend_per_cust'] = raw_df['initial_spend'] / raw_df['N_week_cohort']
    raw_df['rpt_spend_per_cust'] = raw_df['rpt_spend'] / raw_df['N_week_cohort']
        
    raw_df = raw_df[['acq_week', 'week', 'tenure', 'acq_quarter', 'N_week_cohort', 'active_users',
            'merchant_index', 'merchant', 'parent_merchant', 'category', 'subcategory',
            'orders', 'initial_order', 'rpt_orders',
            'orders_per_cust', 'initial_order_per_cust', 'rpt_orders_per_cust',
            'spend', 'initial_spend', 'rpt_spend',
            'spend_per_cust', 'initial_spend_per_cust', 'rpt_spend_per_cust',
            'aov', 'initial_aov', 'rpt_aov'
            ]]

    print(raw_df.shape[0], 'rows')
    raw_df.sort_values(['acq_week', 'week'])

    ### fill missing values in AOV = 0/0
    raw_df[['aov', 'initial_aov', 'rpt_aov']] = raw_df[['aov', 'initial_aov', 'rpt_aov']].fillna(method = 'ffill') # propagate non-null values forward

    # ## Define main columns: {group, time, value}
    # raw_df.rename(columns={'acq_week': 'group', 
    #                 'week': 'time', 
    #                 'order_normalized': 'value'}, inplace=True)

    ## limit data time range
    df = raw_df[ (raw_df['acq_week']<=TEST_END) & (raw_df['acq_week']>=TRAIN_START) ]
    df = df[df['week'] <= TEST_END]

    return df

In [5]:
# ### read acq_initspend data
# cov = pd.read_csv(f'{CURRENT_PATH}/data/preprocessed_data/{DLNAME}/acq_initialspend/company_0_100.csv')

# ### extract covariates
# cov_unique = cov[cov.merchant_index==1090][['acq_week', 'year', 'holidays', 'weekofyear', 'linear', 'linear_1']].drop_duplicates()
# cov_unique = cov_unique.sort_values(['acq_week']).reset_index(drop=True)
# cov_unique 

# ### save covariates
# cov_unique = cov_unique.rename({'acq_week':'week'}, axis=1)
# cov_unique.to_csv(f'{CURRENT_PATH}/utils/covariates_df.csv', index=False)

In [6]:
def generate_covariates(df):

    ### load covariates
    cov = pd.read_csv(f'{CURRENT_PATH}/utils/covariates_df.csv')

    ### merge with current cohort-base data
    data_df = df.merge(cov, left_on='week', right_on='week')
    data_df['firm_cohort_id'] = data_df["merchant_index"].astype(str) + "-" + data_df["acq_week"]
    
    return data_df


In [14]:
### for loop
data_df_list = []
for K in K_LIST:
    df = read_file(K)
    
    if (df[df['acq_week']>=TEST_START].shape[0] != 0):
        data_df_list.append(generate_covariates(df))

### combine everything
df_combined = pd.concat(data_df_list).reset_index(drop=True)

# ### for company_0_100.csv, please remove merchant_index=1028, merchant=lyft_bikes_scooters
# indexdrop = df_combined[df_combined['merchant_index'] == 1028].index
# df_combined.drop(indexdrop , inplace=True)

### combine everything
df_combined = df_combined.sort_values(['merchant_index', 'acq_week', 'week'])
df_combined.to_csv(f'{SAVE_DIR}/company_0_1000.csv', index=True) # must allow index


K = 0 , COMPANY_NAME: 1000_jam_city_inc_jam_city_inc
31125 rows
K = 1 , COMPANY_NAME: 1001_heritage_dairy_heritage_dairy
48828 rows
K = 2 , COMPANY_NAME: 1002_unitedhealth_optumrx
48828 rows
K = 3 , COMPANY_NAME: 1003_h_r_block_h_r_diy
48828 rows
K = 4 , COMPANY_NAME: 1004_bmw_payments_bmw_payments
48828 rows
K = 5 , COMPANY_NAME: 1005_west_marine_west_marine
48828 rows
K = 6 , COMPANY_NAME: 1006_unilever_ben_and_jerries
48828 rows
K = 7 , COMPANY_NAME: 1007_albertsons_adjusted_haggen
44850 rows
K = 8 , COMPANY_NAME: 1008_albertsons_adjusted_star_market
48828 rows
K = 9 , COMPANY_NAME: 1009_marriott_gaylord_hotels
48828 rows
K = 10 , COMPANY_NAME: 100_ahold_delhaize_adjusted_giant_landover
48828 rows
K = 11 , COMPANY_NAME: 1010_aldo_aldo
48828 rows
K = 12 , COMPANY_NAME: 1011_carnival_carnival_cruises
48828 rows
K = 13 , COMPANY_NAME: 1012_foodtown_foodtown
48828 rows
K = 14 , COMPANY_NAME: 1013_ross_dd_discount
48828 rows
K = 15 , COMPANY_NAME: 1014_jane_com_jane_com
48828 rows
K = 16

In [41]:
### attach cohort ID (as an integer)

df_combined = pd.read_csv(f'{SAVE_DIR}/company_0_1000.csv')
df_combined = df_combined.drop(['Unnamed: 0'], axis=1)

cohort_list = np.sort(df_combined['acq_week'].unique())
corhort_dict = {cohort_list[i]:i for i in range(cohort_list.shape[0])}
df_combined['cohort_id'] = df_combined['acq_week'].map(corhort_dict)
df_combined

# df_combined.to_csv(f'{SAVE_DIR}/company_0_1000.csv')

,acq_week,week,tenure,acq_quarter,N_week_cohort,active_users,merchant_index,merchant,parent_merchant,category,subcategory,orders,initial_order,rpt_orders,orders_per_cust,initial_order_per_cust,rpt_orders_per_cust,spend,initial_spend,rpt_spend,spend_per_cust,initial_spend_per_cust,rpt_spend_per_cust,aov,initial_aov,rpt_aov,year,holidays,weekofyear,linear,linear_1,firm_cohort_id,corhort_id
0,2017-01-01,2017-01-01,0,2017-01-01,9840,9840,11,itunes,apple,Home Entertainment,Music Streaming & Audio,11615,9840,1775,1.180386,1.0,0.180386,117724.48080,93803.30760,23921.1732,11.963870,9.532856,2.431014,10.135556,9.532856,13.476717,0.25,1,52,0.068182,0.004649,11-2017-01-01,0
1,2017-01-01,2017-01-08,1,2017-01-01,9840,2071,11,itunes,apple,Home Entertainment,Music Streaming & Audio,2778,0,2778,0.282317,0.0,0.282317,31851.15040,0.00000,31851.1504,3.236906,0.000000,3.236906,11.465497,9.532856,11.465497,0.25,0,1,0.073864,0.005456,11-2017-01-01,0
2,2017-01-01,2017-01-15,2,2017-01-01,9840,1789,11,itunes,apple,Home Entertainment,Music Streaming & Audio,2467,0,2467,0.250711,0.0,0.250711,29201.22320,0.00000,29201.2232,2.967604,0.000000,2.967604,11.836734,9.532856,11.836734,0.25,0,2,0.079545,0.006327,11-2017-01-01,0
3,2017-01-01,2017-01-22,3,2017-01-01,9840,1667,11,itunes,apple,Home Entertainment,Music Streaming & Audio,2223,0,2223,0.225915,0.0,0.225915,25824.77960,0.00000,25824.7796,2.624469,0.000000,2.624469,11.617085,9.532856,11.617085,0.25,0,3,0.085227,0.007264,11-2017-01-01,0
4,2017-01-01,2017-01-29,4,2017-01-01,9840,2865,11,itunes,apple,Home Entertainment,Music Streaming & Audio,3568,0,3568,0.362602,0.0,0.362602,35570.30680,0.00000,35570.3068,3.614869,0.000000,3.614869,9.969256,9.532856,9.969256,0.25,0,4,0.090909,0.008264,11-2017-01-01,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14266326,2020-02-09,2020-02-16,1,2020-01-01,389,10,1100,hotwire_com,expedia_adjusted,Travel & Transportation,Online Travel Agency,12,0,12,0.030848,0.0,0.030848,2270.85000,0.00000,2270.8500,5.837661,0.000000,5.837661,189.237500,211.379831,189.237500,1.00,0,7,0.994318,0.988669,1100-2020-02-09,162
14266327,2020-02-09,2020-02-23,2,2020-01-01,389,2,1100,hotwire_com,expedia_adjusted,Travel & Transportation,Online Travel Agency,3,0,3,0.007712,0.0,0.007712,258.58000,0.00000,258.5800,0.664730,0.000000,0.664730,86.193333,211.379831,86.193333,1.00,0,8,1.000000,1.000000,1100-2020-02-09,162
14266328,2020-02-16,2020-02-16,0,2020-01-01,400,400,1100,hotwire_com,expedia_adjusted,Travel & Transportation,Online Travel Agency,424,400,24,1.060000,1.0,0.060000,86651.71425,83283.79425,3367.9200,216.629286,208.209486,8.419800,204.367251,208.209486,140.330000,1.00,0,7,0.994318,0.988669,1100-2020-02-16,163
14266329,2020-02-16,2020-02-23,1,2020-01-01,400,8,1100,hotwire_com,expedia_adjusted,Travel & Transportation,Online Travel Agency,10,0,10,0.025000,0.0,0.025000,1655.12000,0.00000,1655.1200,4.137800,0.000000,4.137800,165.512000,208.209486,165.512000,1.00,0,8,1.000000,1.000000,1100-2020-02-16,163


In [23]:
### fill missing values again
df_combined = pd.read_csv(f'{SAVE_DIR}/company_0_100.csv')
df_combined = df_combined.drop(['Unnamed: 0'], axis=1)

df_combined[['orders_per_cust', 'initial_order_per_cust', 'rpt_orders_per_cust',
             'spend_per_cust', 'initial_spend_per_cust', 'rpt_spend_per_cust',
             'aov', 'initial_aov', 'rpt_aov',
             ]] = df_combined[['orders_per_cust', 'initial_order_per_cust', 'rpt_orders_per_cust',
             'spend_per_cust', 'initial_spend_per_cust', 'rpt_spend_per_cust',
             'aov', 'initial_aov', 'rpt_aov',
             ]].fillna(method = 'ffill') # propagate non-null values forward

# df_combined.to_csv(f'{SAVE_DIR}/company_0_1000.csv')

In [27]:
df_combined.to_csv(f'{SAVE_DIR}/company_0_100.csv')

In [26]:
print(df_combined[df_combined['rpt_aov'].isnull()].values.any())
# print(df_combined3[df_combined3['rpt_aov'].isnull()])

# df_combined3.iloc[1542486:1542496] ## sanity check

False


In [2]:
df_combined1 = pd.read_csv(f'{SAVE_DIR}/company_0_10.csv')
df_combined1

,Unnamed: 0,acq_week,week,tenure,acq_quarter,N_week_cohort,active_users,merchant_index,merchant,parent_merchant,category,subcategory,orders,initial_order,rpt_orders,orders_per_cust,initial_order_per_cust,rpt_orders_per_cust,spend,initial_spend,rpt_spend,spend_per_cust,initial_spend_per_cust,rpt_spend_per_cust,aov,initial_aov,rpt_aov,year,holidays,weekofyear,linear,linear_1,firm_cohort_id,cohort_id
0,0,2017-03-12,2017-03-12,0,2017-01-01,534,534,1000,jam_city_inc,jam_city_inc,Home Entertainment,Gaming,931,534,397,1.743446,1.0,0.743446,4827.36100,2788.4010,2038.96000,9.040002,5.221725,3.818277,5.185135,5.221725,5.135919,0.25,0,10,0.125000,0.015625,1000-2017-03-12,10
1,1,2017-03-12,2017-03-19,1,2017-01-01,534,344,1000,jam_city_inc,jam_city_inc,Home Entertainment,Gaming,849,0,849,1.589888,0.0,1.589888,4151.51000,0.0000,4151.51000,7.774363,0.000000,7.774363,4.889882,5.221725,4.889882,0.25,0,11,0.130682,0.017078,1000-2017-03-12,10
2,2,2017-03-12,2017-03-26,2,2017-01-01,534,280,1000,jam_city_inc,jam_city_inc,Home Entertainment,Gaming,764,0,764,1.430712,0.0,1.430712,3699.60000,0.0000,3699.60000,6.928090,0.000000,6.928090,4.842408,5.221725,4.842408,0.25,0,12,0.136364,0.018595,1000-2017-03-12,10
3,3,2017-03-12,2017-04-02,3,2017-01-01,534,290,1000,jam_city_inc,jam_city_inc,Home Entertainment,Gaming,705,0,705,1.320225,0.0,1.320225,3661.47000,0.0000,3661.47000,6.856685,0.000000,6.856685,5.193574,5.221725,5.193574,0.25,0,13,0.142045,0.020177,1000-2017-03-12,10
4,4,2017-03-12,2017-04-09,4,2017-01-01,534,262,1000,jam_city_inc,jam_city_inc,Home Entertainment,Gaming,662,0,662,1.239700,0.0,1.239700,3215.25000,0.0000,3215.25000,6.021067,0.000000,6.021067,4.856873,5.221725,4.856873,0.25,0,14,0.147727,0.021823,1000-2017-03-12,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135340,135340,2020-02-09,2020-02-16,1,2020-01-01,547,135,1009,gaylord_hotels,marriott,Travel & Transportation,Lodging & Accommodation,274,0,274,0.500914,0.0,0.500914,38509.92470,0.0000,38509.92470,70.402056,0.000000,70.402056,140.547170,56.161371,140.547170,1.00,0,7,0.994318,0.988669,1009-2020-02-09,162
135341,135341,2020-02-09,2020-02-23,2,2020-01-01,547,3,1009,gaylord_hotels,marriott,Travel & Transportation,Lodging & Accommodation,4,0,4,0.007313,0.0,0.007313,1147.55000,0.0000,1147.55000,2.097898,0.000000,2.097898,286.887500,56.161371,286.887500,1.00,0,8,1.000000,1.000000,1009-2020-02-09,162
135342,135342,2020-02-16,2020-02-16,0,2020-01-01,604,604,1009,gaylord_hotels,marriott,Travel & Transportation,Lodging & Accommodation,948,604,344,1.569536,1.0,0.569536,66627.84235,37580.2600,29047.58235,110.310997,62.218974,48.092024,70.282534,62.218974,84.440646,1.00,0,7,0.994318,0.988669,1009-2020-02-16,163
135343,135343,2020-02-16,2020-02-23,1,2020-01-01,604,74,1009,gaylord_hotels,marriott,Travel & Transportation,Lodging & Accommodation,162,0,162,0.268212,0.0,0.268212,10365.55000,0.0000,10365.55000,17.161507,0.000000,17.161507,63.984877,62.218974,63.984877,1.00,0,8,1.000000,1.000000,1009-2020-02-16,163


In [3]:
df_combined2 = pd.read_csv(f'{SAVE_DIR}/company_0_100.csv')
df_combined2

,Unnamed: 0,acq_week,week,tenure,acq_quarter,N_week_cohort,active_users,merchant_index,merchant,parent_merchant,category,subcategory,orders,initial_order,rpt_orders,orders_per_cust,initial_order_per_cust,rpt_orders_per_cust,spend,initial_spend,rpt_spend,spend_per_cust,initial_spend_per_cust,rpt_spend_per_cust,aov,initial_aov,rpt_aov,year,holidays,weekofyear,linear,linear_1,firm_cohort_id,cohort_id
0,0,2017-01-01,2017-01-01,0,2017-01-01,1695,1695,100,giant_landover,ahold_delhaize_adjusted,Grocers,Supermarkets,1904,1695,209,1.123304,1.0,0.123304,98197.050,82570.460,15626.59,57.933363,48.714136,9.219227,51.574081,48.714136,74.768373,0.25,1,52,0.068182,0.004649,100-2017-01-01,0
1,1,2017-01-01,2017-01-08,1,2017-01-01,1695,225,100,giant_landover,ahold_delhaize_adjusted,Grocers,Supermarkets,328,0,328,0.193510,0.0,0.193510,15162.690,0.000,15162.69,8.945540,0.000000,8.945540,46.227713,48.714136,46.227713,0.25,0,1,0.073864,0.005456,100-2017-01-01,0
2,2,2017-01-01,2017-01-15,2,2017-01-01,1695,231,100,giant_landover,ahold_delhaize_adjusted,Grocers,Supermarkets,357,0,357,0.210619,0.0,0.210619,16784.840,0.000,16784.84,9.902560,0.000000,9.902560,47.016359,48.714136,47.016359,0.25,0,2,0.079545,0.006327,100-2017-01-01,0
3,3,2017-01-01,2017-01-22,3,2017-01-01,1695,206,100,giant_landover,ahold_delhaize_adjusted,Grocers,Supermarkets,296,0,296,0.174631,0.0,0.174631,11634.260,0.000,11634.26,6.863870,0.000000,6.863870,39.304932,48.714136,39.304932,0.25,0,3,0.085227,0.007264,100-2017-01-01,0
4,4,2017-01-01,2017-01-29,4,2017-01-01,1695,204,100,giant_landover,ahold_delhaize_adjusted,Grocers,Supermarkets,293,0,293,0.172861,0.0,0.172861,10831.170,0.000,10831.17,6.390071,0.000000,6.390071,36.966451,48.714136,36.966451,0.25,0,4,0.090909,0.008264,100-2017-01-01,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1342837,1342837,2020-02-09,2020-02-16,1,2020-01-01,635,14,1090,piercing_pagoda,signet_adjusted,Apparel & Accessories,Jewelry & Watches,14,0,14,0.022047,0.0,0.022047,1088.220,0.000,1088.22,1.713732,0.000000,1.713732,77.730000,80.850890,77.730000,1.00,0,7,0.994318,0.988669,1090-2020-02-09,162
1342838,1342838,2020-02-09,2020-02-23,2,2020-01-01,635,3,1090,piercing_pagoda,signet_adjusted,Apparel & Accessories,Jewelry & Watches,3,0,3,0.004724,0.0,0.004724,68.920,0.000,68.92,0.108535,0.000000,0.108535,22.973333,80.850890,22.973333,1.00,0,8,1.000000,1.000000,1090-2020-02-09,162
1342839,1342839,2020-02-16,2020-02-16,0,2020-01-01,590,590,1090,piercing_pagoda,signet_adjusted,Apparel & Accessories,Jewelry & Watches,602,590,12,1.020339,1.0,0.020339,39840.125,39305.815,534.31,67.525636,66.620025,0.905610,66.179610,66.620025,44.525833,1.00,0,7,0.994318,0.988669,1090-2020-02-16,163
1342840,1342840,2020-02-16,2020-02-23,1,2020-01-01,590,8,1090,piercing_pagoda,signet_adjusted,Apparel & Accessories,Jewelry & Watches,8,0,8,0.013559,0.0,0.013559,541.770,0.000,541.77,0.918254,0.000000,0.918254,67.721250,66.620025,67.721250,1.00,0,8,1.000000,1.000000,1090-2020-02-16,163


In [2]:
df_combined3 = pd.read_csv(f'{SAVE_DIR}/company_0_1000.csv')
df_combined3

,Unnamed: 0,acq_week,week,tenure,acq_quarter,N_week_cohort,active_users,merchant_index,merchant,parent_merchant,category,subcategory,orders,initial_order,rpt_orders,orders_per_cust,initial_order_per_cust,rpt_orders_per_cust,spend,initial_spend,rpt_spend,spend_per_cust,initial_spend_per_cust,rpt_spend_per_cust,aov,initial_aov,rpt_aov,year,holidays,weekofyear,linear,linear_1,firm_cohort_id,cohort_id
0,0,2017-01-01,2017-01-01,0,2017-01-01,9840,9840,11,itunes,apple,Home Entertainment,Music Streaming & Audio,11615,9840,1775,1.180386,1.0,0.180386,117724.48080,93803.30760,23921.1732,11.963870,9.532856,2.431014,10.135556,9.532856,13.476717,0.25,1,52,0.068182,0.004649,11-2017-01-01,0
1,1,2017-01-01,2017-01-08,1,2017-01-01,9840,2071,11,itunes,apple,Home Entertainment,Music Streaming & Audio,2778,0,2778,0.282317,0.0,0.282317,31851.15040,0.00000,31851.1504,3.236906,0.000000,3.236906,11.465497,9.532856,11.465497,0.25,0,1,0.073864,0.005456,11-2017-01-01,0
2,2,2017-01-01,2017-01-15,2,2017-01-01,9840,1789,11,itunes,apple,Home Entertainment,Music Streaming & Audio,2467,0,2467,0.250711,0.0,0.250711,29201.22320,0.00000,29201.2232,2.967604,0.000000,2.967604,11.836734,9.532856,11.836734,0.25,0,2,0.079545,0.006327,11-2017-01-01,0
3,3,2017-01-01,2017-01-22,3,2017-01-01,9840,1667,11,itunes,apple,Home Entertainment,Music Streaming & Audio,2223,0,2223,0.225915,0.0,0.225915,25824.77960,0.00000,25824.7796,2.624469,0.000000,2.624469,11.617085,9.532856,11.617085,0.25,0,3,0.085227,0.007264,11-2017-01-01,0
4,4,2017-01-01,2017-01-29,4,2017-01-01,9840,2865,11,itunes,apple,Home Entertainment,Music Streaming & Audio,3568,0,3568,0.362602,0.0,0.362602,35570.30680,0.00000,35570.3068,3.614869,0.000000,3.614869,9.969256,9.532856,9.969256,0.25,0,4,0.090909,0.008264,11-2017-01-01,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14266136,14266326,2020-02-09,2020-02-16,1,2020-01-01,389,10,1100,hotwire_com,expedia_adjusted,Travel & Transportation,Online Travel Agency,12,0,12,0.030848,0.0,0.030848,2270.85000,0.00000,2270.8500,5.837661,0.000000,5.837661,189.237500,211.379831,189.237500,1.00,0,7,0.994318,0.988669,1100-2020-02-09,162
14266137,14266327,2020-02-09,2020-02-23,2,2020-01-01,389,2,1100,hotwire_com,expedia_adjusted,Travel & Transportation,Online Travel Agency,3,0,3,0.007712,0.0,0.007712,258.58000,0.00000,258.5800,0.664730,0.000000,0.664730,86.193333,211.379831,86.193333,1.00,0,8,1.000000,1.000000,1100-2020-02-09,162
14266138,14266328,2020-02-16,2020-02-16,0,2020-01-01,400,400,1100,hotwire_com,expedia_adjusted,Travel & Transportation,Online Travel Agency,424,400,24,1.060000,1.0,0.060000,86651.71425,83283.79425,3367.9200,216.629286,208.209486,8.419800,204.367251,208.209486,140.330000,1.00,0,7,0.994318,0.988669,1100-2020-02-16,163
14266139,14266329,2020-02-16,2020-02-23,1,2020-01-01,400,8,1100,hotwire_com,expedia_adjusted,Travel & Transportation,Online Travel Agency,10,0,10,0.025000,0.0,0.025000,1655.12000,0.00000,1655.1200,4.137800,0.000000,4.137800,165.512000,208.209486,165.512000,1.00,0,8,1.000000,1.000000,1100-2020-02-16,163


In [55]:
df_combined3[df_combined3['merchant_index']==484]

,Unnamed: 0,acq_week,week,tenure,acq_quarter,N_week_cohort,active_users,merchant_index,merchant,parent_merchant,category,subcategory,orders,initial_order,rpt_orders,orders_per_cust,initial_order_per_cust,rpt_orders_per_cust,spend,initial_spend,rpt_spend,spend_per_cust,initial_spend_per_cust,rpt_spend_per_cust,aov,initial_aov,rpt_aov,year,holidays,weekofyear,linear,linear_1,firm_cohort_id,cohort_id
6144493,6144493,2019-10-20,2019-10-20,0,2019-10-01,12457,12457,484,cumberland_farms,eg_group_adjusted,General Merchandise,Gas Stations,20977,12457,8520,1.683953,1.0,0.683953,427094.9827,282837.2074,144257.7753,34.285541,22.705082,11.580459,20.360156,22.705082,16.931664,0.75,0,42,0.897727,0.805914,484-2019-10-20,146
6144494,6144494,2019-10-20,2019-10-27,1,2019-10-01,12457,7321,484,cumberland_farms,eg_group_adjusted,General Merchandise,Gas Stations,17728,0,17728,1.423136,0.0,1.423136,332915.0573,0.0000,332915.0573,26.725139,0.000000,26.725139,18.779053,22.705082,18.779053,0.75,0,43,0.903409,0.816148,484-2019-10-20,146
6144495,6144495,2019-10-20,2019-11-03,2,2019-10-01,12457,7305,484,cumberland_farms,eg_group_adjusted,General Merchandise,Gas Stations,17714,0,17714,1.422012,0.0,1.422012,348712.4388,0.0000,348712.4388,27.993292,0.000000,27.993292,19.685697,22.705082,19.685697,0.75,0,44,0.909091,0.826446,484-2019-10-20,146
6144496,6144496,2019-10-20,2019-11-10,3,2019-10-01,12457,7059,484,cumberland_farms,eg_group_adjusted,General Merchandise,Gas Stations,16805,0,16805,1.349041,0.0,1.349041,328065.8435,0.0000,328065.8435,26.335863,0.000000,26.335863,19.521919,22.705082,19.521919,0.75,0,45,0.914773,0.836809,484-2019-10-20,146
6144497,6144497,2019-10-20,2019-11-17,4,2019-10-01,12457,7156,484,cumberland_farms,eg_group_adjusted,General Merchandise,Gas Stations,17152,0,17152,1.376897,0.0,1.376897,337867.4588,0.0000,337867.4588,27.122699,0.000000,27.122699,19.698429,22.705082,19.698429,0.75,0,46,0.920455,0.847237,484-2019-10-20,146
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6144678,6144678,2020-02-09,2020-02-16,1,2020-01-01,1292,141,484,cumberland_farms,eg_group_adjusted,General Merchandise,Gas Stations,179,0,179,0.138545,0.0,0.138545,4224.5762,0.0000,4224.5762,3.269796,0.000000,3.269796,23.600984,22.318991,23.600984,1.00,0,7,0.994318,0.988669,484-2020-02-09,162
6144679,6144679,2020-02-09,2020-02-23,2,2020-01-01,1292,83,484,cumberland_farms,eg_group_adjusted,General Merchandise,Gas Stations,98,0,98,0.075851,0.0,0.075851,2078.6400,0.0000,2078.6400,1.608854,0.000000,1.608854,21.210612,22.318991,21.210612,1.00,0,8,1.000000,1.000000,484-2020-02-09,162
6144680,6144680,2020-02-16,2020-02-16,0,2020-01-01,1284,1284,484,cumberland_farms,eg_group_adjusted,General Merchandise,Gas Stations,1425,1284,141,1.109813,1.0,0.109813,32381.2048,29141.5186,3239.6862,25.219007,22.695887,2.523120,22.723652,22.695887,22.976498,1.00,0,7,0.994318,0.988669,484-2020-02-16,163
6144681,6144681,2020-02-16,2020-02-23,1,2020-01-01,1284,127,484,cumberland_farms,eg_group_adjusted,General Merchandise,Gas Stations,160,0,160,0.124611,0.0,0.124611,3144.0881,0.0000,3144.0881,2.448667,0.000000,2.448667,19.650551,22.695887,19.650551,1.00,0,8,1.000000,1.000000,484-2020-02-16,163


In [60]:
### attach cohort ID (as an integer)

df_combined3 = pd.read_csv(f'{SAVE_DIR}/company_0_1000.csv')
df_combined3 = df_combined.drop(['Unnamed: 0'], axis=1)

### remove company which has no training period
TEST_START = '2019-04-01' # Q2 2019
TRAIN_END=(pd.to_datetime(TEST_START) - timedelta(days=1)).strftime('%Y-%m-%d')
temp = df_combined3[df_combined3['merchant_index']==484]
temp[temp['acq_week']<=TRAIN_END].shape[0] # = 0

df_combined3 = df_combined3[df_combined3['merchant_index']!=484]

# df_combined3.to_csv(f'{SAVE_DIR}/company_0_1000.csv')


0

In [70]:
df_combined3.to_csv(f'{SAVE_DIR}/company_0_1000.csv')

In [46]:
### sanity check
print(df_combined.groupby(['merchant_index'])['merchant_index'].count())
df_combined

merchant_index
11      13695
12      13695
13      13695
15      13695
20      13695
        ...  
1096    13695
1097    13695
1098    13695
1099    13695
1100    13695
Name: merchant_index, Length: 1061, dtype: int64


,acq_week,week,tenure,acq_quarter,N_week_cohort,active_users,merchant_index,merchant,parent_merchant,category,subcategory,orders,initial_order,rpt_orders,orders_per_cust,initial_order_per_cust,rpt_orders_per_cust,spend,initial_spend,rpt_spend,spend_per_cust,initial_spend_per_cust,rpt_spend_per_cust,aov,initial_aov,rpt_aov,year,holidays,weekofyear,linear,linear_1,firm_cohort_id,cohort_id
0,2017-01-01,2017-01-01,0,2017-01-01,9840,9840,11,itunes,apple,Home Entertainment,Music Streaming & Audio,11615,9840,1775,1.180386,1.0,0.180386,117724.48080,93803.30760,23921.1732,11.963870,9.532856,2.431014,10.135556,9.532856,13.476717,0.25,1,52,0.068182,0.004649,11-2017-01-01,0
1,2017-01-01,2017-01-08,1,2017-01-01,9840,2071,11,itunes,apple,Home Entertainment,Music Streaming & Audio,2778,0,2778,0.282317,0.0,0.282317,31851.15040,0.00000,31851.1504,3.236906,0.000000,3.236906,11.465497,9.532856,11.465497,0.25,0,1,0.073864,0.005456,11-2017-01-01,0
2,2017-01-01,2017-01-15,2,2017-01-01,9840,1789,11,itunes,apple,Home Entertainment,Music Streaming & Audio,2467,0,2467,0.250711,0.0,0.250711,29201.22320,0.00000,29201.2232,2.967604,0.000000,2.967604,11.836734,9.532856,11.836734,0.25,0,2,0.079545,0.006327,11-2017-01-01,0
3,2017-01-01,2017-01-22,3,2017-01-01,9840,1667,11,itunes,apple,Home Entertainment,Music Streaming & Audio,2223,0,2223,0.225915,0.0,0.225915,25824.77960,0.00000,25824.7796,2.624469,0.000000,2.624469,11.617085,9.532856,11.617085,0.25,0,3,0.085227,0.007264,11-2017-01-01,0
4,2017-01-01,2017-01-29,4,2017-01-01,9840,2865,11,itunes,apple,Home Entertainment,Music Streaming & Audio,3568,0,3568,0.362602,0.0,0.362602,35570.30680,0.00000,35570.3068,3.614869,0.000000,3.614869,9.969256,9.532856,9.969256,0.25,0,4,0.090909,0.008264,11-2017-01-01,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14266326,2020-02-09,2020-02-16,1,2020-01-01,389,10,1100,hotwire_com,expedia_adjusted,Travel & Transportation,Online Travel Agency,12,0,12,0.030848,0.0,0.030848,2270.85000,0.00000,2270.8500,5.837661,0.000000,5.837661,189.237500,211.379831,189.237500,1.00,0,7,0.994318,0.988669,1100-2020-02-09,162
14266327,2020-02-09,2020-02-23,2,2020-01-01,389,2,1100,hotwire_com,expedia_adjusted,Travel & Transportation,Online Travel Agency,3,0,3,0.007712,0.0,0.007712,258.58000,0.00000,258.5800,0.664730,0.000000,0.664730,86.193333,211.379831,86.193333,1.00,0,8,1.000000,1.000000,1100-2020-02-09,162
14266328,2020-02-16,2020-02-16,0,2020-01-01,400,400,1100,hotwire_com,expedia_adjusted,Travel & Transportation,Online Travel Agency,424,400,24,1.060000,1.0,0.060000,86651.71425,83283.79425,3367.9200,216.629286,208.209486,8.419800,204.367251,208.209486,140.330000,1.00,0,7,0.994318,0.988669,1100-2020-02-16,163
14266329,2020-02-16,2020-02-23,1,2020-01-01,400,8,1100,hotwire_com,expedia_adjusted,Travel & Transportation,Online Travel Agency,10,0,10,0.025000,0.0,0.025000,1655.12000,0.00000,1655.1200,4.137800,0.000000,4.137800,165.512000,208.209486,165.512000,1.00,0,8,1.000000,1.000000,1100-2020-02-16,163
